In [116]:
#chrome.exe --remote-debugging-port=9222 --user-data-dir="C:/Chrome_debug/"
#https://tix.ctbcsports.com/DEA/UTK0101_

#先使用上面 新開chorme 點到要購買的物品位置
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# 設定 Chrome 選項，連接到遠端調試端口
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

# 連接到已啟動的 Chrome 瀏覽器
driver = webdriver.Chrome(options=chrome_options)

#選日期
specific_td = driver.find_element(By.XPATH, '//td[.//div[contains(text(), "2024/11/27 19:00")]]')
specific_td.click()

time.sleep(1)

# 使用 JavaScript 點擊
buy_btn = driver.find_element(By.ID, 'buy_btn')
driver.execute_script("arguments[0].click();", buy_btn)

time.sleep(1)


seat = driver.find_element(By.XPATH, '//td[contains(text(), "B6普通席")]')
seat.click()

# 等待操作完成
time.sleep(0.5)

green = driver.find_element(By.XPATH, '//button[contains(@class, "f1 green")]')
green.click()


time.sleep(0.5)
# 定義所有排數的選項，例如 'B', 'C', 'D', ...
rows = ['D', 'E', 'F', 'G', 'H', 'I', 'J','K','L']

#locationChoice = driver.find_element(By.XPATH, '//div[contains(@class, "locationChoice")]')

#driver.execute_script("arguments[0].scrollIntoView();", locationChoice)
for row in rows:
    start_seat_number = 59  # 開始的座位號
    num_seats_to_select = 4  # 需要選擇的連號座位數量
    max_seat_number = 90  # 假設每排座位最多到 60 號
    
    current_seat = start_seat_number
    while current_seat <= max_seat_number:
        seats_to_select = []
        
        # 嘗試抓取該排的連號座位
        for i in range(current_seat, current_seat + num_seats_to_select*2,2):
            xpath = f'//td[contains(@title, "B6普通席-{row}排-{i}號")]'
            try:
                # 找到座位
                td_element = driver.find_element(By.XPATH, xpath)
                
                # 將可用座位加入到列表中
                seats_to_select.append(td_element)
                print(len(seats_to_select))
            except NoSuchElementException:
                # 如果有一個座位不存在或無法選擇，跳過這一組座位
                seats_to_select = []
                print(f"座位不存在: {xpath}")
                break  # 退出內層循環
        
        # 如果成功抓到 4 個連號座位，點擊它們
        if len(seats_to_select) == num_seats_to_select:
            for seat in seats_to_select:
                seat.click()
                time.sleep(0.5)  # 適當的延遲
            break  # 成功選到4個連號，退出外層循環
        
        # 如果沒選到這組座位，移到下一組
        current_seat += 2  # 注意注意注意!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!  如果是 1 3 5 7 要設定成2
    
    # 如果成功找到座位，就不用再檢查其他排了
    if len(seats_to_select) == num_seats_to_select:
        break

code  = driver.find_element(By.XPATH, '//img[@id="chk_pic"]')
driver.execute_script("arguments[0].scrollIntoView();", code)

time.sleep(1)

driver.save_screenshot('test.png')

# 打開圖片
img = Image.open('test.png')

# 獲取圖片尺寸
width, height = img.size

# 設定裁剪邊界
left = 740
upper = 390
right = min(877, width)  # 保證不超出範圍
lower = min(430, height)  # 保證不超出範圍

# 裁剪圖片
img_cropped = img.crop((left, upper, right, lower))
import ddddocr
# 保存裁剪後的圖片
img_cropped.save('cropped_image.png')
ocr = ddddocr.DdddOcr()

with open('cropped_image.png','rb') as f:
    img_bytes = f.read()

text = ocr.classification(img_bytes)
upper_text = text.upper()

print(upper_text)

captcha_input = driver.find_element(By.ID, 'CHK') 
captcha_input.send_keys(upper_text)

addcart = driver.find_element(By.ID, 'addcart') 
addcart.click()

time.sleep(2)

# 抓取對話框元素
dialog = driver.find_element(By.XPATH, '//div[contains(@role, "dialog")]')

# 檢查該對話框是否顯示
display_value = dialog.value_of_css_property('display')
while(display_value != 'none'):
    time.sleep(1)
    buttons = driver.find_elements(By.TAG_NAME, 'button')
    for button in buttons:
        if button.text == 'Ok':
            button.click()  # 點擊按鈕
            break
    driver.execute_script("arguments[0].scrollIntoView();", code)
    time.sleep(1)
    driver.save_screenshot('test1.png')

    # 打開圖片
    img = Image.open('test.png')
    
    # 獲取圖片尺寸
    width, height = img.size
    
    # 設定裁剪邊界
    left = 740
    upper = 390
    right = min(877, width)  # 保證不超出範圍
    lower = min(430, height)  # 保證不超出範圍
    
    # 裁剪圖片
    img_cropped = img.crop((left, upper, right, lower))
    # 保存裁剪後的圖片
    img_cropped.save('cropped_image1.png')
    ocr = ddddocr.DdddOcr()
    
    with open('cropped_image1.png','rb') as f:
        img_bytes = f.read()
    
    text = ocr.classification(img_bytes)
    upper_text = text.upper()
    captcha_input.clear()
    time.sleep(1)
    captcha_input.send_keys(upper_text)    
    addcart.click()

# 關閉瀏覽器
driver.quit()


1
2
3
4
欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
PWGR


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=130.0.6723.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7E7B5F0B5+27685]
	(No symbol) [0x00007FF7E7AC5E10]
	(No symbol) [0x00007FF7E796581A]
	(No symbol) [0x00007FF7E7977CFB]
	(No symbol) [0x00007FF7E796CF24]
	(No symbol) [0x00007FF7E796D0A9]
	(No symbol) [0x00007FF7E796AE29]
	(No symbol) [0x00007FF7E796E91F]
	(No symbol) [0x00007FF7E79FCBD0]
	(No symbol) [0x00007FF7E79DBA6A]
	(No symbol) [0x00007FF7E79FB8FC]
	(No symbol) [0x00007FF7E79DB813]
	(No symbol) [0x00007FF7E79A75E8]
	(No symbol) [0x00007FF7E79A8751]
	GetHandleVerifier [0x00007FF7E7E53FCD+3128125]
	GetHandleVerifier [0x00007FF7E7EA4EF0+3459680]
	GetHandleVerifier [0x00007FF7E7E9AFFD+3418989]
	GetHandleVerifier [0x00007FF7E7C26EAB+846363]
	(No symbol) [0x00007FF7E7